In [60]:
import os
import sys
import json
from scipy.integrate import solve_ivp
from scipy.stats import ttest_rel
import numpy as np
from numpy.linalg import norm
from numpy import sqrt
from math import pi
import pickle
import matplotlib.pyplot as plt
from packages import data_container
from packages.data_container import Data
from packages.helper import play_trajs, rotate, sp2a, v2sp, dist, psi, beta, d_theta, d_psi, sp2v, dist, \
    traj_speed, min_sep
from packages.ode_simulator import ODESimulator
from fitting import Cohen_movObst2, Cohen_movObst1
from sklearn.metrics import mean_squared_error

# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Cohen_movObst1_data.pickle'))
with open(file, 'rb') as f:
    data = pickle.load(f)

'''Fajen_approach1a'''
fajen_approach = {'name': 'fajen_approach', 'b1': 2.01938384, 'k1': 4.90527274, 'c1': 2.96094879, 'c2': 0.50896525,
                  'k2': 1.61216734}
fajen_approach2 = {'name': 'fajen_approach2', 'b1': 2.04992354, 'k1': 2.85641543, 'c1': 0.54294928, 'c2': 0.73857217,
                   'b2': 3.89580222, 'k2': 5.04511601}

acceleration_approach = {'name': 'acceleration_approach', 'k': 0.9220813483746707}
#--------------------------------------------------------------------------------------

'''Cohen_movObst1'''

# cohen_avoid = {'name': 'cohen_avoid', 'b1': 2.1260796879288923, 'k1': 673.0529452234115, 'c5': 14.968106833780706,
#                'c6': 1.0453773247966653, 'b2': 0.006893708430404716, 'k2': 13.305731910646147, 'c7': 6.887725262876937,
#                'c8': 0.626517752901824}

# cohen_avoid2 = {'name': 'cohen_avoid2', 'b1': 1.313894780845967, 'k1': 393.59559030933247, 'c5': 8.82965005381204,
#                 'c6': 2.3410827991863172, 'b2': 0.0003485317797959908, 'k2': 66.77017554542931, 'c7': 7.602409489606929,
#                 'c8': 1.5994854927685118}

# cohen_avoid3 = {'name': 'cohen_avoid3', 'k1': 40.758773179123104, 'c5': 9.73184518588978, 'c6': 23.508572875262164,
#                 'k2': 35.65382695736378, 'c7': 9.814413003581466, 'c8': 20.776807046770987}

# cohen_avoid4 = {'name': 'cohen_avoid4', 'k1': 4.798312212313739, 'c5': 1.4381497616822931, 'c6': 5.126613085209973,
#                 'k2': 18.94607374676705, 'c7': 8.103786608747624, 'c8': 2.167764111347458}

# perpendicular_avoid2 = {'name': 'perpendicular_avoid2', 'k': 5.355486629738685, 'c': 15.073121867842529}

#-----------------------------------------------------------------------------------------

'''Cohen_movObst2'''
cohen_avoid = {'name': 'cohen_avoid', 'b1': 3.363638816353943, 'k1': 95.09517044816117, 'c5': 11.400136527121575,
               'c6': 0.48808376780737783, 'b2': 0.0, 'k2': 594.7235062798342, 'c7': 4.535696669262642,
               'c8': 3.777451780362486}

cohen_avoid2 = {'name': 'cohen_avoid2', 'b1': 3.3485438586491, 'k1': 119.59655224362285, 'c5': 8.320041684111693,
                'c6': 9.59390592684502, 'b2': 0.0, 'k2': 16.022521094515888, 'c7': 3.714647768807139,
                'c8': 3.971298762659632}

cohen_avoid3 = {'name': 'cohen_avoid3', 'k1': 27.015824466499474, 'c5': 6.439764681301262, 'c6': 28.71370803318366,
                'k2': 23.09740785255075, 'c7': 3.6069831344642873, 'c8': 13.659355893212165}

cohen_avoid4 = {'name': 'cohen_avoid4', 'k1': 2.0736814583750123, 'c5': 0.277869777057664, 'c6': 11.55562093696106,
                'k2': 0.695174380649082, 'c7': 5.707392197706118, 'c8': 7.709406732831272}
#-----------------------------------------------------------------------------------------

In [ ]:
'''Simulation with var0'''
%matplotlib qt
Hz = 100
xg0, yg0, xo0, yo0, x0, y0 = 0, 10, 5, 5, 0, 0
vxo0, vyo0, vx0, vy0 = -1, 0, 0, 1
s0, phi0 = v2sp([vx0, vy0])
dphi0 = dds = 0
var0 = [xg0, yg0, xo0, yo0, vxo0, vyo0, x0, y0, vx0, vy0, phi0, s0, dphi0, dds]
models = [fajen_approach, cohen_avoid]
args = {'w_goal': 0.1, 'w_obst': 0.1}
sim = ODESimulator(Hz=90, models=models, args=args, ref=[0, 1])
sim.simulate(var0, total_time=10)
sim.play()

In [2]:
'''Simulate one trial'''
############
i_trial = 861
approach = fajen_approach
avoid = cohen_avoid

############
%matplotlib qt
models = [approach, avoid]
sim = ODESimulator(data=data, ref=[0, 1])
sim.models = models
sim.simulate_all(trials=[i_trial], t_start='obst_onset', t_end='obst_out', ps='trial')
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
sim.test('p_dist')
sim.play(title=title)

Loading finished
solve_ivp crashed. Switch to Euler method
Simulated 1 trials in 0:00:00


In [65]:
'''Simuate all trials model 1'''
############
# subject = 1
approach = fajen_approach
avoid = cohen_avoid2
############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim1, trials1 = Cohen_movObst2(subjects)
sim1.reset()
sim1.models = models

# for i in range(len(data.trajs)):
#     if i in data.dump or data.info['subj_id'][i] != subject or data.info['obst_speed'][i] == 0:
#         continue
#     trials.append(i)
sim1.simulate_all(trials1, t_start='obst_onset', t_end='obst_out', ps='trial')
print(sim1.test('p_dist'))
print(sim1.test('order_accuracy'))

Loading finished
Simulated 875 trials in 0:00:27
0.24663529942639034
0.7851428571428571


In [66]:
'''Simuate all trials model 2'''
############
# subject = 1
approach = fajen_approach2
avoid = cohen_avoid4

# avoid =  {'name': 'cohen_avoid4', 'k1': 19.548, 'c5': 0.116, 'c6': 24.700, 'k2': 91.621, 'c7': 5.009, 'c8': 16.079}
# avoid =  {'name': 'cohen_avoid4', 'k1': 81.396, 'c5': 0.518, 'c6': 18.739, 'k2': 87.143, 'c7': 1.879, 'c8': 27.955}
# avoid =  {'name': 'cohen_avoid4', 'k1': 56.617, 'c5': 0.136, 'c6': 9.106, 'k2': 58.301, 'c7': 7.441, 'c8': 1.563}
# avoid =  {'name': 'cohen_avoid4', 'k1': 82.222, 'c5': 0.186, 'c6': 6.506, 'k2': 12.226, 'c7': 7.056, 'c8': 23.840}
# avoid =  {'name': 'cohen_avoid4', 'k1': 37.100, 'c5': 0.657, 'c6': 7.380, 'k2': 52.451, 'c7': 3.455, 'c8': 23.441}


############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim2, trials2 = Cohen_movObst2(subjects)
sim2.reset()
sim2.models = models

# for i in range(len(data.trajs)):
#     if i in data.dump or data.info['subj_id'][i] != subject or data.info['obst_speed'][i] == 0:
#         continue
#     trials.append(i)
sim2.simulate_all(trials2, t_start='obst_onset', t_end='obst_out', ps='trial')
print(sim2.test('p_dist'))
print(sim2.test('order_accuracy'))

Loading finished
Simulated 875 trials in 0:00:22
0.24684128959749277
0.7885714285714286


In [68]:
'''BIC'''
sim = sim1
k = 8
n = len(sim.p_pred)
MSE = sim.test('p_MSE')*2
BIC = n * np.log(MSE) + k * np.log(n)
print(f'The BIC of model {sim.models[1]["name"]} is {BIC}, MSE is {MSE}, k is {k}, n is {n}')

The BIC of model cohen_avoid2 is -1541.9730813728042, MSE is 0.16134816854221473, k is 8, n is 875


In [60]:
'''Find certain errors'''
sim = sim1
errors = sim.test('p_dist', all_errors=True)
print(f'smallest error {np.argmin(errors)}')
for i, err in enumerate(errors):
    if 0.189 < err < 0.19:
        print(i)

smallest error 158
663
897


In [63]:
'''Play simulations'''
sim = sim1
i_trial = 897
print(sim.test('p_dist', i_trials=[i_trial]))
sim.play(i_trial=i_trial, save=True)


0.18927789389401092


In [5]:
'''Check simulation length'''
ls = []
for i in range(len(sim.p_pred)):
    ls.append(len(sim.p_pred[i]))
np.mean(ls)/30

2.991859296482412

In [37]:
'''Speed and heading errors correct vs incorrect trials'''
from sklearn.metrics import mean_squared_error
sim = sim1
phi_MSEs = []
speed_MSEs = []
trials = []
for i in range(len(sim.v_pred)):
    j = sim.i_trials[i]
#     if sim.pass_order_pred[i] == sim.data.info['pass_order'][j]:
    v_subj = np.gradient(sim.p_subj[i], axis=0) * sim.data.Hz
    v_pred = sim.v_pred[i]
    s0, phi0 = v2sp(v_subj)
    s1, phi1 = v2sp(v_pred)
    speed_MSEs.append(mean_squared_error(s0, s1))
    phi_MSEs.append(mean_squared_error(phi0, phi1))
    trials.append(i)
print(sim.models)
print(f'{len(phi_MSEs)} trials in total')
print(f'RMSE on heading is {np.sqrt(np.mean(phi_MSEs))}')
print(f'RMSE on speed is {np.sqrt(np.mean(speed_MSEs))}')
print(f"Distance is {sim.test('p_dist', i_trials=trials)}")

[{'name': 'fajen_approach', 'b1': 2.01938384, 'k1': 4.90527274, 'c1': 2.96094879, 'c2': 0.50896525, 'k2': 1.61216734, 'ps': 1.1242147210583378}, {'name': 'cohen_avoid2', 'b1': 3.3485438586491, 'k1': 119.59655224362285, 'c5': 8.320041684111693, 'c6': 9.59390592684502, 'b2': 0.0, 'k2': 16.022521094515888, 'c7': 3.714647768807139, 'c8': 3.971298762659632, 'ps': 1.1242147210583378}]
875 trials in total
RMSE on heading is 0.23135130693377928
RMSE on speed is 0.09203448273536953
Distance is 0.24663529942639034


In [ ]:
sim2.test('p_dist')

In [ ]:
'''Minimum Passing Distance'''
%matplotlib qt
sim = sim2
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title('Signed predicted minimum passing distance (SMPD)')
ax.set_ylabel('SMPD (m)')
ax.set_xlabel('normalized time (%)')
ax.set_ylim((-2, 2))
for j, i in enumerate(sim.i_trials):
    t0 = data.info['obst_onset'][i]
    t1 = data.info['obst_out'][i]
    p0 = sim.p_pred[j]
    p1 = data.info['p_obst'][i][t0:t1]
    v0 = sim.v_pred[j]
    v1 = data.info['v_obst'][i][t0:t1]
    t = np.linspace(0, 100, len(p0))
    smpd = []
    for _p0, _p1, _v0, _v1 in zip(p0, p1, v0, v1):        
        smpd.append(min_sep(_p0, _p1, _v0, _v1)[0])
    ax.plot(t, smpd, 'k', linewidth=0.1, alpha=0.5)

In [ ]:
print(len(p0), len(p1), len(sim.i_trials))

In [ ]:
'''Compute pred_order'''
s = sim
orders = []
for i in range(len(s.p_pred)):
    j = s.i_trials[i]
    true_order = s.data.info['pass_order'][j]
    p0 = s.p_pred[i][-1]
    v0 = s.v_pred[i][-1]
    p1 = s.p_obst[i][-1]
    _beta = beta(p0, p1, v0)
    angle = s.data.info['obst_angle'][j]
    pred_order = np.sign(_beta * -angle)
    orders.append(true_order == pred_order)
print(np.mean(orders))

In [ ]:
sim.test('order_accuracy')

In [ ]:
'''t-test between two models'''
err1 = sim.test('p_dist', all_errors=True)
err2 = sim2.test('p_dist', all_errors=True)
print(ttest_rel(err1, err2))
print(f'sd1 = {np.std(err1)}, sd2 = {np.std(err2)}')
print(f'df = {len(err1) - 1}')

In [ ]:
'''Show one trial from batch simulations'''
######## index in simulated trials only
i = 83
########
i_trial = sim.i_trials[i]
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
sim.play(i, title=title, save=False)

# When beta and dpsi has the same sign it means pass in front, otherwise it means pass from behind
pass_order = sim.data.info['pass_order'][i]
pred = sim.pass_order_pred[i]
print('pass order ', pass_order, 'prediction ', pred)
print(f"err is {sim.test('p_dist', i_trial=i)}")

In [23]:
'''plot true vs predicted speed by condition'''
#####################
subject = 1
#####################
%matplotlib qt
Hz = sim.Hz
fig = plt.figure()
fig.suptitle('Subject ' + str(subject))
axes = {}
obst_angle = sorted(set([abs(x) for x in data.info['obst_angle'] if x != 0]))
obst_speed = sorted(set([abs(x) for x in data.info['obst_speed'] if x != 0]))
i_plot = 1
for angle in obst_angle:
    for speed in obst_speed:
        axes[(angle, speed)] = fig.add_subplot(5, 5, i_plot)
        axes[(angle, speed)].set_xlim(0, 5)
        axes[(angle, speed)].set_ylim(0.5, 2)
        axes[(angle, speed)].set_title(str(angle) + '° ' + str(speed) + 'm/s')
#         axes[(angle, speed)].set_aspect('equal')
        i_plot += 1
for i, i_trial in enumerate(sim.i_trials):
    speed = sim.data.info['obst_speed'][i_trial]
    angle = sim.data.info['obst_angle'][i_trial]
    subj_id = sim.data.info['subj_id'][i_trial]
    if subj_id != subject or speed == 0:
        continue
    
    subj, pred = sim.p_subj[i], sim.p_pred[i]
    s_subj, s_pred = traj_speed(subj, Hz), traj_speed(pred, Hz)
    t = np.linspace(0, len(pred)-1, len(pred)) / Hz
    # Speed
    axes[(abs(angle), speed)].plot(t, s_subj, 'r')
    axes[(abs(angle), speed)].plot(t, s_pred, 'b')


In [22]:
'''plot true vs predicted heading by condition'''
#####################
subject = 1
#####################
%matplotlib qt
Hz = sim.Hz
fig = plt.figure()
fig.suptitle('Subject ' + str(subject))
axes = {}
obst_angle = sorted(set([abs(x) for x in data.info['obst_angle'] if x != 0]))
obst_speed = sorted(set([abs(x) for x in data.info['obst_speed'] if x != 0]))
i_plot = 1
for angle in obst_angle:
    for speed in obst_speed:
        axes[(angle, speed)] = fig.add_subplot(5, 5, i_plot)
        axes[(angle, speed)].set_xlim(0, 5)
        axes[(angle, speed)].set_ylim(0, 1.5)
        axes[(angle, speed)].set_title(str(angle) + '° ' + str(speed) + 'm/s')
#         axes[(angle, speed)].set_aspect('equal')
        i_plot += 1
for i, i_trial in enumerate(sim.i_trials):
    speed = sim.data.info['obst_speed'][i_trial]
    angle = sim.data.info['obst_angle'][i_trial]
    subj_id = sim.data.info['subj_id'][i_trial]
    if subj_id != subject or speed == 0:
        continue
    
    subj, pred = sim.p_subj[i], sim.p_pred[i]
    h_subj, h_pred = v2sp(np.gradient(subj, axis=0) * Hz)[1], v2sp(np.gradient(pred, axis=0) * Hz)[1]
    if h_subj[0] < 0:
        h_subj += pi
        h_pred += pi
    t = np.linspace(0, len(pred)-1, len(pred)) / Hz
    # Heading
    axes[(abs(angle), speed)].plot(t, h_subj, 'r')
    axes[(abs(angle), speed)].plot(t, h_pred, 'b')
    if abs(angle) == 112.5 and speed == 1.2:
        print(i, i_trial)


In [42]:
'''Cross validation error'''
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Results', 'cohen_movObst2_cross_validation_cohen_avoid'))
filenames = [os.path.join(path, name) for name in os.listdir(path) if name[-3:] == 'txt']
MSEs = {}
for filename in filenames:
    with open(filename, 'rb') as f:
        best = 0
        e_min = float('inf')
        if filename[-6] == '_' or filename[-6] == '-':
            subj_id = int(filename[-5])
        else:
            subj_id = int(filename[-6:-4])
        for i, line in enumerate(f):
            if i >= 5:
                try:
                    err = str(line).split("\\t")[-4][:10]
                    if err[0] == '0':
                        err = float(err)
                        if err < e_min:
                            e_min = err
                            best = str(line).replace("\\", "")
                except:
                    pass
    try:
        models = json.loads(best.split('t')[1].replace("\'", "\""))
        subjects = [int(subj_id)]
        sim, trials = Cohen_movObst2(subjects)
        sim.reset()
        sim.models = models
        sim.simulate_all(trials, t_start='obst_onset', t_end='obst_out', ps='trial')
        MSEs[subj_id] = sim.test('p_dist', all_errors=True)
        print(f'{subj_id} finished')
    except:
        pass

Loading finished
Simulated 59 trials in 0:00:01
1 finished
Loading finished
Simulated 56 trials in 0:00:01
2 finished
Loading finished
Simulated 58 trials in 0:00:01
3 finished
Loading finished
Simulated 60 trials in 0:00:01
4 finished
Loading finished
Simulated 60 trials in 0:00:02
5 finished
Loading finished
Simulated 56 trials in 0:00:01
6 finished
Loading finished
Simulated 59 trials in 0:00:02
7 finished
Loading finished
Simulated 58 trials in 0:00:02
10 finished
Loading finished
Simulated 59 trials in 0:00:02
11 finished
Loading finished
Simulated 58 trials in 0:00:01
12 finished
Loading finished
Simulated 60 trials in 0:00:01
13 finished
Loading finished
Simulated 57 trials in 0:00:01
14 finished
Loading finished
Simulated 56 trials in 0:00:01
15 finished
Loading finished
Simulated 59 trials in 0:00:01
8 finished
Loading finished
Simulated 60 trials in 0:00:01
9 finished


In [43]:
len(data.trajs)/15

60.0

In [43]:
for key, val in MSEs.items():
    print(key, np.mean(val))

1 0.4227768353884516
2 0.15307592873592307
3 0.17925545108523722
4 0.37783747862651024
5 0.2600656831531423
6 0.17447361605719194
7 0.46172314029079636
10 0.18032311731628467
11 0.17418104839082255
12 0.1882962978540945
13 0.2118958955885841
14 0.3693317240573899
15 0.2941238468642867
8 0.23790223787787934
9 0.14174547610042862
